In [19]:
import pandas as pd
import os
import glob

In [5]:
# Get the current working directory
current_path = os.getcwd()
print("Current Path:", current_path)

# Set a new working directory
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)


Current Path: /Users/yifanli/Github/fidelity-portfolio-tracker/code


In [35]:
# Define the folder path containing the files
folder_path = './data'

# Use glob to get a list of all CSV files that match the pattern
file_pattern = os.path.join(folder_path, 'Accounts_History_*.csv')
csv_files = glob.glob(file_pattern)

# Read each file as a DataFrame and add to a list
dataframes = [pd.read_csv(file, usecols=range(13)) for file in csv_files]
dataframes = [dat[dat['Account'].notna()] for dat in dataframes]

# Concatenate all DataFrames by rows
combined_df = pd.concat(dataframes, ignore_index=True)

# # Display the combined DataFrame
print(combined_df.head())

      Run Date                                 Account  \
0   09/29/2023                    Individual Z23390746   
1   09/29/2023  Cash Management (Individual) Z06872898   
2   09/29/2023                    Individual Z23390746   
3   09/26/2023                    Individual Z23390746   
4   09/19/2023                    Individual Z23390746   

                                              Action  Symbol  \
0   REINVESTMENT FIDELITY TREASURY MONEY MARKET F...   FZFXX   
1   INTEREST EARNED FDIC INSURED DEPOSIT AT US BA...   QUSBQ   
2   DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...   FZFXX   
3   YOU BOUGHT NIKE INC CLASS B COM NPV (NKE) (Cash)     NKE   
4   YOU BOUGHT NIKE INC CLASS B COM NPV (NKE) (Cash)     NKE   

                                Description  Type  Quantity  Price ($)  \
0       FIDELITY TREASURY MONEY MARKET FUND  Cash     50.75        1.0   
1   FDIC INSURED DEPOSIT AT US BANK NOT COV  Cash      0.00        NaN   
2       FIDELITY TREASURY MONEY MARKET FUND 

In [36]:
combined_df

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
0,09/29/2023,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,50.75,1.0,NaN,NaN,NaN,-50.75,NaN
1,09/29/2023,Cash Management (Individual) Z06872898,INTEREST EARNED FDIC INSURED DEPOSIT AT US BA...,QUSBQ,FDIC INSURED DEPOSIT AT US BANK NOT COV,Cash,0.00,NaN,NaN,NaN,NaN,0.23,NaN
2,09/29/2023,Individual Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,50.75,NaN
3,09/26/2023,Individual Z23390746,YOU BOUGHT NIKE INC CLASS B COM NPV (NKE) (Cash),NKE,NIKE INC CLASS B COM NPV,Cash,10.00,90.0,NaN,NaN,NaN,-900.00,09/28/2023
4,09/19/2023,Individual Z23390746,YOU BOUGHT NIKE INC CLASS B COM NPV (NKE) (Cash),NKE,NIKE INC CLASS B COM NPV,Cash,10.00,95.0,NaN,NaN,NaN,-950.00,09/21/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,11/01/2022,Individual Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.00,150.0,NaN,NaN,NaN,-1500.00,11/03/2022
312,10/31/2022,Individual Z23390746,DIVIDEND RECEIVED JPMORGAN CHASE &CO. COM (JP...,JPM,JPMORGAN CHASE &CO. COM,Cash,0.00,NaN,NaN,NaN,NaN,10.00,NaN
313,10/31/2022,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,43.45,1.0,NaN,NaN,NaN,-43.45,NaN
314,10/31/2022,Cash Management (Individual) Z06872898,INTEREST EARNED FDIC INSURED DEPOSIT AT US BA...,QUSBQ,FDIC INSURED DEPOSIT AT US BANK NOT COV,Cash,0.00,NaN,NaN,NaN,NaN,0.13,NaN
